# 1 Introduction

The applications of recommender systems in businesses have become increasingly popular. Recommender systems apply various sources of information including demographics, social, and preferences to provide users with tailored recommended items. Moreover, the favoured technique for building recommender systems is Collaborative filtering. This technique is further divided into three main categories including Memory-Based, Model-Based, and Hybrid-BAsed. Therefore, as large businesses realise the advantages of recommending personalized items to users, the research for techniques, sources of information, and implementation grows. Netflix is among the big organisations interested in the expansion of recommmender system. The application of recommender systems at Netflix is widely known, however improvements are continiously being investigated to provide users with the best movie and series recommendations. Thus, this notebook aims to investigate the application of collaborative filtering techniques for Netflix.      

#### Which type of RecSys based on CF could Netflix user to provide the most accurate recommendations to users?

- What are the different type of RecSys based on CF?
- Which types could be used for Netflix' dataset?
- How do KNN and SVD compare?

# 2 Importing Libraries

Importing the necessary libraries

In [753]:
# General imports
import os
import pandas as pd
import numpy as np

# Used for visualisations in the EDA
import seaborn as sns

# Used in reducing the memory storage of sparse matrices
from scipy.sparse import csr_matrix

# Used for creating a KNN and SVD RecSys model
from sklearn.model_selection import train_test_split
from sklearn.neighbors import NearestNeighbors
from scipy.sparse.linalg import svds

# Used for performance evaluation
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error

# As working with NaN values in matrices overwelmed the output with warnings 
# these warnings will be ignored.
import warnings
warnings.filterwarnings('ignore')

# 3 Data Processing

## 3.1 Netflix Dataset

### 3.1.1 Import Source Data

Code to append movieId to each record in all of the source files if this has not been executed earlier. This will allow all the source files to be loaded into a dataframe with one line of code and without having to add the movieId seperately before concatting the sourcefiles.

In [754]:
def format_netflix_source():
    x = 0
    string = ","+str(x)

    # Loop through each file in the directory
    for filename in os.listdir(directory):
        with open(os.path.join(directory, filename), 'r') as f:
            # Check if the first line of the first file already has been formatted
            # If yes the formatting will be skipped
            if(f.readlines()[0] == '1:,1\n'):
                print('Source is already formatted, continuing \n')
                return
            else:
                print('Started formatting source files \n')
                # Append a ',', movieId and \n (newline) to each line 
                file_lines = [''.join([x.strip(), string, '\n']) for x in f.readlines()]
        with open(os.path.join(directory, filename), 'w') as f:
            # Save the formatted file
            f.writelines(file_lines)
            print('Completed formatting source files')
    return

Creating the movie dataframe by concatting all the sourcefiles without their title (skiprows=1). Concluding with naming the columns.

In [755]:
#
directory = "/Users/vbraun/Downloads/training_set/"

# 
format_netflix_source()

print('Started concatting all source files to DataFrame \n')
# Importing all different files into one dataframe (including itemId)
# As the files do not have a header and differ between each other 'header' = None and the first row will be skipped
netflix_df = pd.concat(pd.read_csv(os.path.join(directory, fname), skiprows=1,header=None) for fname in os.listdir(directory)).rename(columns={0:'userId',1:'rating',2:'date',3:'itemId'})
print('Completed concatting all source files to DataFrame \n')

# Dropping the date column as this is not relevant for this recommender system
netflix_df = netflix_df.drop(columns='date')

display(netflix_df.head(3))

### 3.1.2 Data Filtering

To allow for faster development a debugging variable is used. If debugging is True the dataset will only consist of the first 100 movies. For the final model, debugging will be set to False.

In [ ]:
debugging = True
# 
if debugging == True:
    print('Debugging is set to True: limiting the dataset to the first 100 movies')
    filtered_netflix_df = netflix_df[netflix_df['itemId'] <= 100]
else:
    filtered_netflix_df = netflix_df
print('Length of selected dataset: {0} \n'.format(len(filtered_netflix_df)))

In order to filter the dataset based on activity and reduce the sparsity of the data, the data will be grouped and filtered based on movies and users. The datasets will show how many ratings each movie has gotten and how many rating each user has given.

To reduce the sparcity of data in the dataset, we will filter out the users that have rated fewer than 5% of the total amount of movies.

Finally, the movies that have been rated by fewer than 50 people will be filtered out of the dataset.

In [756]:
# 
filtered_movie_count = filtered_netflix_df[['itemId','userId']].groupby('itemId').count().reset_index().rename(columns={'userId':'user_count'})
filtered_user_count = filtered_netflix_df[['itemId','userId']].groupby('userId').count().reset_index().rename(columns={'itemId':'item_count'})
# display(filtered_movie_count.head(3),filtered_user_count.head(3))

required_rated_percentage = 0.05
print('Filtering out users that have rated less than {0}% of all items'.format(int(required_rated_percentage*100)))
# Look if the userId exists when the total amount of items rated by a user (filtered_user_count['item_count']), divided by the total amount of movies (len(filtered_movie_count)' is bigger than the required_rated_percentage (default = 0.05)
filtered_netflix_df = filtered_netflix_df[filtered_netflix_df['userId'].isin(filtered_user_count[filtered_user_count['item_count']/len(filtered_movie_count) > required_rated_percentage]['userId'])]
print('Length of filtered dataset: {0} \n'.format(len(filtered_netflix_df)))

print('Filtering out movies that have been rated by fewer than {0} users'.format(50))
# 
filtered_netflix_df = filtered_netflix_df[filtered_netflix_df['itemId'].isin(filtered_movie_count[filtered_movie_count['user_count']>50]['itemId'])]
print('Length of filtered dataset:',len(filtered_netflix_df))

## 3.2 Jester Dataset

### 3.2.1 Import Source Data

In [757]:
jester_items = pd.read_csv(r'C:\Users\vbraun\Downloads\SDM-Datasets\jester_items.csv')
jester_ratings = pd.read_csv(r'C:\Users\vbraun\Downloads\SDM-Datasets\jester_ratings.csv')

jester_df = jester_ratings.rename(columns={'jokeId':'itemId'})
jester_df['rating'] = jester_df['rating'] + 10

### 3.2.2 Data filtering

In [758]:
jester_item_count = jester_df[['itemId','userId']].groupby('itemId').count().reset_index().rename(columns={'userId':'user_count'})
jester_user_count = jester_df[['itemId','userId']].groupby('userId').count().reset_index().rename(columns={'itemId':'item_count'})

required_rated_percentage = 0.05
filtered_jester_df = jester_df[jester_df['userId'].isin(jester_user_count[jester_user_count['item_count']/len(jester_item_count) > required_rated_percentage]['userId'])]
filtered_jester_df = filtered_jester_df[filtered_jester_df['itemId'].isin(jester_item_count[jester_item_count['user_count']>20]['itemId'])]

# 4 Exploratory Data Analysis (EDA)

## 4.1 EDA Netflix Dataset

In [759]:
# print('The filtered dataset has', filtered_df['userId'].nunique(), 'unique users')
# print('The filtered dataset has', filtered_df['itemId'].nunique(), 'unique movies')
# print('The filtered dataset has', filtered_df['rating'].nunique(), 'unique ratings')
# print('The unique ratings are', sorted(filtered_df['rating'].unique()))

In [760]:
# display(filtered_df.head(),filtered_df.tail())

In [761]:
# filtered_df.describe()

In [762]:
# print('Amount of NaN values in the dataset:',filtered_df.loc[lambda x: x.isnull().any(axis=1)].shape[0])

The following graph shows for each movie (as a dot) what its mean rating is in comparison to the total amount of ratings. 

In [763]:
# plt = sns.jointplot(x='rating_mean', y='rating_amount', data=filtered_df.groupby('itemId').agg(rating_mean = ('rating', 'mean'), rating_amount = ('rating', 'count')).reset_index())
# plt.fig.suptitle("Rating Mean and Rating Amount of Movies")
# plt.fig.subplots_adjust(top=0.95)

Notably, movies with low mean ratings have generally been rated a low number of times in relation to movies with a mean rating higher than 3.0.

The next graph shows the same variables as the graph seen above for each user. 

In [764]:
# plt = sns.jointplot(x='rating_mean', y='rating_amount', data=filtered_df.groupby('userId').agg(rating_mean = ('rating', 'mean'), rating_amount = ('rating', 'count')).reset_index())
# plt.fig.suptitle("Rating Mean and Rating Amount of Users")
# plt.fig.subplots_adjust(top=0.95)

The graph for users show that there are several outliers of users that have rated many movies while having a low mean of their ratings. Additionally, the graph shows that mean of the rating mean approaches normality. 

## 4.2 EDA Jester Dataset

In [ ]:
# print('The filtered dataset has', filtered_df['userId'].nunique(), 'unique users')
# print('The filtered dataset has', filtered_df['itemId'].nunique(), 'unique movies')
# print('The filtered dataset has', filtered_df['rating'].nunique(), 'unique ratings')
# print('The unique ratings are', sorted(filtered_df['rating'].unique()))

In [ ]:
# display(filtered_df.head(),filtered_df.tail())

In [ ]:
# filtered_df.describe()

In [ ]:
# print('Amount of NaN values in the dataset:',filtered_df.loc[lambda x: x.isnull().any(axis=1)].shape[0])

In [ ]:
# plt = sns.jointplot(x='rating_mean', y='rating_amount', data=filtered_df.groupby('itemId').agg(rating_mean = ('rating', 'mean'), rating_amount = ('rating', 'count')).reset_index())
# plt.fig.suptitle("Rating Mean and Rating Amount of Movies")
# plt.fig.subplots_adjust(top=0.95)

In [ ]:
# plt = sns.jointplot(x='rating_mean', y='rating_amount', data=filtered_df.groupby('userId').agg(rating_mean = ('rating', 'mean'), rating_amount = ('rating', 'count')).reset_index())
# plt.fig.suptitle("Rating Mean and Rating Amount of Users")
# plt.fig.subplots_adjust(top=0.95)

# 5 Recommender Systems

In [766]:
def evaluate_predictions(pred, truth):
    pred = pred[truth.nonzero()].flatten()
    truth = truth[truth.nonzero()].flatten()

    rmse = np.sqrt(mean_squared_error(pred,truth))
    mae = mean_absolute_error(pred,truth)
    
    return rmse, mae

## 5.1 K Nearest Neighbors (KNN)

### 5.1.1 KNN Model

In [791]:
def train_test(filtered_df):

    sparse_matrix = csr_matrix(filtered_df.pivot_table(index='userId', columns='itemId', values='rating').fillna(0).values)
    sparse_matrix.check_format

    train_data, test_data = train_test_split(sparse_matrix, test_size=.10)
    test_data, validation_data = train_test_split(test_data, test_size=.50)

    return train_data, validation_data, test_data

In [771]:
def calculate_knn_prediction(train_data, test_data, k = 5, metric = 'cosine', n_neighbors = 20):
    knn_model = NearestNeighbors(metric=metric,algorithm='brute',n_neighbors=n_neighbors,n_jobs=-1)

    knn_model_fitted = knn_model.fit(train_data.toarray())
    distance, indices = knn_model_fitted.kneighbors(test_data.toarray(),k)

    raw_recommends = sorted(list(zip(indices.squeeze().tolist(), distance.squeeze().tolist())), key=lambda x: x[1])[:0:-1]
    knn_prediction = []
    for i, (idx, dist) in enumerate(raw_recommends):
        td = pd.DataFrame(train_data.toarray())
        sim_users = np.array(td[td.index.isin(idx)])
        sim_users[sim_users == 0] = np.nan
        average_rat = np.nan_to_num(np.nanmean(sim_users,axis=0))
        knn_prediction.append(average_rat)
    
    # rmse, mae = evaluate_predictions(test_data.toarray(),np.array(knn_prediction))
    
    return np.array(knn_prediction)


### 5.1.2 KNN Hyper Parameter Tuning

In [789]:
def hyper_parameter_tuning_knn(train_data, validation_data):
    n_neighbors = [5,10,20,50]
    recommendation_amount = [3,5,10]
    metric = ['euclidean','manhattan','cosine','minkowski']

    hpt_results = []
    for met in metric:
        for k in recommendation_amount:
            for n in n_neighbors:
                rmse, mae = evaluate_predictions(validation_data.toarray(),calculate_knn_prediction(train_data = train_data, test_data = validation_data, metric = met, k = k, n_neighbors = n))
                print(rmse,met,k,n)
                hpt_results.append([rmse,met,k,n])

    best_parameters_knn = sorted(hpt_results, key=lambda x: x[0])[0]
    print(best_parameters)

    return best_parameters_knn

1. Create NearestNeighbors model
1. Fit the model with train data
1. Use kneighbors to find the k amount of neighbors of the jokes in the test data
1. Calculate the prediction by taking the average score of the k most similar jokes
1. Evaluate the model by comparing the actual ratings with the predicted ratings

Algorithm is set at brute (force) because the inputdata is sparse

## 5.2 Singular Value Decomposition (SVD)

### 5.2.1 SVD Model

Pivot the dataset into a matrix with index='userId', columns='itemId', values='rating' in order to later perform user-based collaborative filtering. Moreover, fill_value = 0 in order to remove NaN values and save them as 0. Finally, the matrix is directly stored as a sparse matrix to save memory, instead of first saving the entire matrix into memory. 

Scipy.sparse.linalg.svds was used to perform a partial singular value decomposition of a sparse matrix. This function allows us to specify 'k' which is the number of singular values and singular vectors that have to be computed. 

In [765]:
def calculate_svd_prediction(data, k = 5):
    # Performing the SVD matrix factorisation giving: u (m x r) orthogonal matrix, 
    # s (r x r) diagonal matrix, and vt(ransposed) (r x n) orthogonal matrix.
    u, s, vt = svds(data.toarray(), k = k)

    # A diagonal matrix has to be created for s in order to recreate a matrix from u, s, and vt
    s_diagonal = np.diag(s)

    # Recreate the matrix by performing matrix multiplications of u, s, and vt
    predictions = np.dot(np.dot(u, s_diagonal), vt)
    
    return predictions

In order to evaluate the performance of the recommendations following SVD we only need the $\hat{y}$ of existing $y$. Therefore, all other values will be filtered out of the prediction dataset by using pred[truth.nonzero()]. Afterwards we are able to evaluate the performance of our model by comparing $\hat{y}$ with their corresponding $y$. 

### 5.2.2 SVD Hyper Parameter Tuning

Different k values lead to different predictions

We will perform hyperparameter tuning to find the k with the lowest rmse. For each k we will perform multiple iterations, in which a random sample of the data will be masked and used to calculate the rmse. The rmse of a k value will be the average rmse of all iterations of that k. 

In [767]:
def hyper_parameter_tuning_svd(dataset, iterations = 10):
    results = []

    print('Calculating the average rmse over {0} iterations'.format(iterations))

    k_list = [1,2,3,4,5,6,7,8,9,10,20,30,50,80]

    for k in k_list:
        rmse_list = []
        for i in range(0,iterations):
            dataset_ex_masked, masked_data = train_test_split(dataset, test_size=.05)

            dataset_ex_masked_csr = csr_matrix(dataset_ex_masked.pivot_table(index='userId', columns='itemId', values='rating').fillna(0).values)
            masked_data_csr = csr_matrix(masked_data.pivot_table(index='userId', columns='itemId', values='rating').fillna(0).values)

            rmse, mae = evaluate_predictions(calculate_svd_prediction(dataset_ex_masked_csr,k),masked_data_csr.toarray())
            
            rmse_list.append(rmse)
    
        results.append([k,(sum(rmse_list)/len(rmse_list))])
        print('For k = {0}, the average rmse = {1}'.format(k,(sum(rmse_list)/len(rmse_list))))

    best_parameters_svd = sorted(results, key=lambda x: x[1])[0]
    print('The rmse is lowest for k = {0} at = {1}'.format(best_parameters_svd[0],best_parameters_svd[1]))
    
    return best_parameters_svd
    

## SVD Recommendation

In [768]:

# final_csr_matrix = csr_matrix(filtered_df.pivot_table(index='userId', columns='itemId', values='rating').fillna(0).values)
# predictions = hyper_svd(final_csr_matrix,best_parameters_svd[0])

In [769]:
# recommend_for_user = 650

# user_pred_df = pd.DataFrame(predictions)
# user_sel_pred_df = user_pred_df.loc[recommend_for_user].sort_values(ascending=False)

# user_df = pd.DataFrame(final_csr_matrix.toarray())
# selected = pd.DataFrame(user_df.loc[recommend_for_user])
# rated_movies = selected.loc[~(selected==0).all(axis=1)].index.values.tolist()

# recommended_movies = user_sel_pred_df.loc[~user_sel_pred_df.index.isin(rated_movies)]

# print('Rated items are:',rated_movies)
# print(recommended_movies[:3])

# 6 Model Evaluation

## 6.1 Performance Netflix Dataset

### 6.1.1 Results KNN model

In [777]:
knn_train, knn_validation, knn_test = train_test(filtered_netflix_df)

In [778]:
best_params_knn = hyper_parameter_tuning_knn(train_data = knn_train, validation_data = knn_validation)

euclidean
manhattan
cosine
minkowski
[2.4038460040135923, 'euclidean', 10, 5]


In [779]:
evaluate_predictions(knn_test.toarray(),calculate_knn_prediction(train_data = knn_train, test_data = knn_test, metric = best_params_knn[1], k = best_params_knn[2], n_neighbors = best_params_knn[3]))

(2.4111006751666118, 2.1145233715352387)

### 6.1.2 Results SVD model

In [776]:
best_params_svd = hyper_parameter_tuning_svd(dataset = filtered_netflix_df, iterations = 10)

Calculating the average rmse over 10 iterations
For k = 1, the average rmse = 3.3015731898684115
For k = 2, the average rmse = 3.2872733279246105
For k = 3, the average rmse = 3.242653750720451
For k = 4, the average rmse = 3.294263396501619
For k = 5, the average rmse = 3.300066123582045
For k = 6, the average rmse = 3.3099805502183357
For k = 7, the average rmse = 3.286975095455766
For k = 8, the average rmse = 3.2822585245805724
For k = 9, the average rmse = 3.2961219122878553
For k = 10, the average rmse = 3.284985285116142
For k = 20, the average rmse = 3.329788566274631
For k = 30, the average rmse = 3.3324355402642007
For k = 50, the average rmse = 3.3649706393756134
For k = 80, the average rmse = 3.388405900587698
The rmse is lowest for k = 3 at = 3.242653750720451


### 6.1.3 Comparison KNN & SVD for Netflix

#### 6.1.3.1 Performance

#### 6.1.3.2 Recommendations

## 6.2 Performance Jester Dataset

### 6.2.1 Results KNN model

In [792]:
knn_train, knn_validation, knn_test = train_test(filtered_jester_df)

In [821]:
best_params_knn = hyper_parameter_tuning_knn(train_data = knn_train, validation_data = knn_validation)

10.151903699790237 euclidean 3 5
10.151903699790237 euclidean 3 10
10.151903699790237 euclidean 3 20
10.151903699790237 euclidean 3 50
10.014150049430668 euclidean 5 5
10.014150049430668 euclidean 5 10
10.014150049430668 euclidean 5 20
10.014150049430668 euclidean 5 50
9.82456844739558 euclidean 10 5
9.82456844739558 euclidean 10 10
9.82456844739558 euclidean 10 20
9.82456844739558 euclidean 10 50
10.731054298178492 manhattan 3 5
10.731054298178492 manhattan 3 10
10.731054298178492 manhattan 3 20
10.731054298178492 manhattan 3 50
10.626282985091194 manhattan 5 5
10.626282985091194 manhattan 5 10
10.626282985091194 manhattan 5 20
10.626282985091194 manhattan 5 50
10.516305141287347 manhattan 10 5
10.516305141287347 manhattan 10 10
10.516305141287347 manhattan 10 20
10.516305141287347 manhattan 10 50
11.122161818474266 cosine 3 5
11.122161818474266 cosine 3 10
11.122161818474266 cosine 3 20
11.122161818474266 cosine 3 50
11.049417775495579 cosine 5 5
11.049417775495579 cosine 5 10
11.049

In [822]:
evaluate_predictions(knn_test.toarray(),calculate_knn_prediction(train_data = knn_train, test_data = knn_test, metric = best_params_knn[1], k = best_params_knn[2], n_neighbors = best_params_knn[3]))

(9.882721365179828, 8.638029386025508)

### 6.2.2 Results SVD model

In [783]:
best_params_svd = hyper_parameter_tuning_svd(dataset = filtered_jester_df, iterations = 2)

Calculating the average rmse over 2 iterations
For k = 1, the average rmse = 10.197680846160697
For k = 2, the average rmse = 10.177194623331173
For k = 3, the average rmse = 10.28298939596786
For k = 4, the average rmse = 10.320629616808938
For k = 5, the average rmse = 10.365160655511655
For k = 6, the average rmse = 10.381040518946396
For k = 7, the average rmse = 10.384607354708603
For k = 8, the average rmse = 10.430343480008583
For k = 9, the average rmse = 10.469413237252866
For k = 10, the average rmse = 10.452437841139657
For k = 20, the average rmse = 10.642962232244766
For k = 30, the average rmse = 10.809716983765423
For k = 50, the average rmse = 10.992329252704964
For k = 80, the average rmse = 11.064206293507606
The rmse is lowest for k = 2 at = 10.177194623331173


In [ ]:
recommend_for_user = 50

user_pred_df = pd.DataFrame(predictions)
user_sel_pred_df = user_pred_df.loc[recommend_for_user].sort_values(ascending=False)

user_df = pd.DataFrame(final_csr_matrix.toarray())
selected = pd.DataFrame(user_df.loc[recommend_for_user])
rated_movies = selected.loc[~(selected==0).all(axis=1)].index.values.tolist()

recommended_movies = user_sel_pred_df.loc[~user_sel_pred_df.index.isin(rated_movies)]

print('Rated items are:',rated_movies)
print(recommended_movies[:3])

Rated items are: [15, 17, 27, 57, 67, 87]
29    1.630473
47    1.303044
32    1.288816
Name: 50, dtype: float64


### 6.2.3 Comparison KNN & SVD for Jester

#### 6.1.3.1 Performance

#### 6.1.3.2 Recommendations

In order to answer the subquestion: "How do the KNN and SVD models compare?" we will compare compare the rmse of both models for the same dataset.

# 7 Conclusion

#### Which type of RecSys based on CF could Netflix user to provide the most accurate recommendations to users?

- How do KNN and SVD compare?